In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cbpro
import datetime
from datetime import datetime as dt
import numpy as np
import time
import pickle
import configparser

### Get product list

In [2]:
## Bring in cbpro public client
public_client = cbpro.PublicClient()
products = public_client.get_products()

In [3]:
product_names = []
for i in products:
    product_names.append(i['id'])

In [4]:
refined = ["BTC-USD","ETH-USD","ETH-BTC"] 

In [16]:
data_1 = pd.read_csv(r"C:\Users\hlu\repos\coinbase_v2\data\60\minute_last_100_days_ETH-USD.csv",index_col = 0)


In [17]:
data_2 = pd.read_csv(r"C:\Users\hlu\repos\coinbase_v2\data\60\minute_last_100_days_ETH-BTC.csv",index_col = 0)
data_3 = pd.read_csv(r"C:\Users\hlu\repos\coinbase_v2\data\60\minute_last_100_days_BTC-USD.csv",index_col = 0)

In [28]:
base_data = pd.DataFrame(data_1['time'])
base_data = base_data.merge(data_1[['time','close']],how="left",on="time")
base_data.rename(columns = {"close":"ETH-USD"},inplace=True)
base_data = base_data.merge(data_2[['time','close']],how="left",on="time")
base_data.rename(columns = {"close":"ETH-BTC"},inplace=True)
base_data = base_data.merge(data_3[['time','close']],how="left",on="time")
base_data.rename(columns = {"close":"BTC-USD"},inplace=True)

In [34]:
base_data['return_ETH_to_USD'] = (base_data['ETH-USD']/ base_data['BTC-USD'])/base_data['ETH-BTC']

In [35]:
base_data['return_BTC_to_USD'] = (base_data['BTC-USD']/ base_data['ETH-USD'])*base_data['ETH-BTC']

In [68]:
n1 = len(base_data[base_data['return_BTC_to_USD']-1>0.003])/len(base_data)

In [69]:
n2 = len(base_data[base_data['return_ETH_to_USD']-1>0.003])/len(base_data)

In [70]:
60*24*365*n1 #Number of successful trades ina  year

255.5266173559746

In [59]:
60*24*365*n2 #Number of successful trades ina  year

178.86863214918222

In [60]:
trade_data= base_data[(base_data['return_ETH_to_USD']-1>0.003)|(base_data['return_BTC_to_USD']-1>0.003)]

In [77]:
trade_data['return'] = np.abs(trade_data['return_ETH_to_USD']-1) - 0.002

<ipython-input-77-93a77abd31f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_data['return'] = np.abs(trade_data['return_ETH_to_USD']-1) - 0.002


In [79]:
trade_data['return'].sum()

0.30549447547574304

In [73]:
len(base_data[base_data['return_ETH_to_USD']-1>0.003])/len(base_data)

0.0003403132270722645

In [22]:
def split_instrument(df,instrument,splitter):
    strings = df[instrument][0].split(splitter)
    df["i1"],df['i2'] = strings[0],strings[1]
    return df

In [23]:
data1 = pd.DataFrame(columns = ['price','instrument'])
data1['price'] = np.random.normal(10,size=1000)
data1['instrument'] = "BTC-USDT"
data1 = split_instrument(data1,"instrument","-")

In [24]:
data2 = pd.DataFrame(columns = ['price','instrument'])
data2['price'] = np.random.normal(10,size=1000)
data2['instrument'] = "ETH-USDT"
data2 = split_instrument(data2,"instrument","-")

In [25]:
data3 = pd.DataFrame(columns = ['price','instrument'])
data3['price'] = np.random.normal(10,size=1000)
data3['instrument'] = "BTC-ETH"
data3 = split_instrument(data3,"instrument","-")
data3['price'] = (data1['price']/data2['price']) -0.01

In [26]:
data = [data1['instrument'][0],data2['instrument'][0],data3['instrument'][0]]

In [30]:
data

['BTC-USDT', 'ETH-USDT', 'BTC-ETH']

In [27]:
first_pairs = []
second_pairs = []
for i in data:
    if "BTC" in i:
        first_pairs.append(i)
    if "USDT" in i:
        second_pairs.append(i)


In [28]:
def three_pairs(a,b,c): #a is asset1/currency, b is asset 2/currency, c is asset 1 / asset 2
    difference = a/b - c
    pct_dif = difference / c
    # If the pct dif is positive, this implies the price of asset 1 in terms of asset 2 is greater when transacting directly
    # rather than going through USDT. This means asset 1 should be bought with currency, sold directly for asset 2,
    # which in turn should be used to re-purchase currency
    
    #If the pct dif is negative, this implies the price of asset 1 in terms of asset 2 is lower when transacting directly
    # rather than going through USDT. This means asset 2 should be bought with currency, sold directly for asset 1, which
    # in turn should be used to re-purchase currency
    return pct_dif
    

In [29]:
three_pairs(data1['price'],data2['price'],data3['price'])

0      0.012390
1      0.009840
2      0.009774
3      0.008167
4      0.010520
         ...   
995    0.015002
996    0.010344
997    0.009490
998    0.009591
999    0.007515
Name: price, Length: 1000, dtype: float64

In [ ]:
def trade(df,signal):
    while signal>0:
        #sell ETH for USDT -100ETH
        #Buy BTC for USDT +200BTC
        # Sell BTC for ETH direct -100BTC, +100 ETH ; net up 100BTC

In [77]:
#a positive number indicates the price via usdt of B in terms of A is higher than in the A B market
# Therefore, we should take B, convert it to usdt then sellusdt for A.We should then sell A for B in the direct market, and repeat


,price,instrument,i1,i2
0,10.913011,BTC-USDT,BTC,USDT
1,9.602708,BTC-USDT,BTC,USDT
2,10.479483,BTC-USDT,BTC,USDT
3,7.481865,BTC-USDT,BTC,USDT


In [78]:
data2.head(4)

,price,instrument,i1,i2
0,6.980550,ETH-USDT,ETH,USDT
1,8.376340,ETH-USDT,ETH,USDT
2,11.076664,ETH-USDT,ETH,USDT
3,8.902408,ETH-USDT,ETH,USDT


In [79]:
data3.head(4)

,price,instrument,i1,i2
0,12.754485,BTC-ETH,BTC,ETH
1,13.705003,BTC-ETH,BTC,ETH
2,10.011291,BTC-ETH,BTC,ETH
3,9.677894,BTC-ETH,BTC,ETH
